# Notebook 01: Data Collection

query = '''DROP TABLE track_list;'''
execute_query(query, command=True)

In [1]:
import json, time, re, requests, spotipy
import pandas as pd
import psycopg2 as pg2

from bs4 import BeautifulSoup
from time import sleep
from spotipy.oauth2 import SpotifyClientCredentials

from psycopg2.extras import RealDictCursor, Json
from psycopg2.extensions import AsIs
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

In [2]:
%run ../assets/sql_cred.py
%run ../assets/spotify_cred.py

In [3]:
client_credential_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credential_manager)

## Configure Postgres Server with Docker

Define functions to programmatically connect to and insert data into database:
-  **con_cur_to_db**: returns both a connection and a cursor object for database
-  **execute_query**: executes query directly to database, without having to create a cursor and connection each time
-  **insert_entry_json**: inserts data into database

In [4]:
def con_cur_to_db(dbname=DBNAME, dict_cur=None):
    con = pg2.connect(host=IP_ADDRESS,
                  dbname=dbname,
                  user=USER,
                  password=PASSWORD)
    if dict_cur:
        cur = con.cursor(cursor_factory=RealDictCursor)
    else:
        cur = con.cursor()
    return con, cur
    
def execute_query(query, dbname=DBNAME, dict_cur=None, command=False):
    con, cur = con_cur_to_db(dbname, dict_cur)
    cur.execute(f'{query}')
    if not command:
        data = cur.fetchall()
        con.close()
        return data
    con.commit() #sends to server
    con.close() #closes server connection

Create table `track_list` to save collected data.

query = '''CREATE TABLE track_list (
track_id VARCHAR PRIMARY KEY,
playlist_id VARCHAR,
track_name VARCHAR,
artist_name VARCHAR,
album_name VARCHAR,
playlist_name VARCHAR,
playlist_owner VARCHAR,
lyrics JSON
)
;'''

execute_query(query, command=True)

In [5]:
def insert_playlist_info(track_dict):
    con, cur = con_cur_to_db()
    
    columns = track_dict.keys()
    values = [track_dict[column] for column in columns]

    insert_statement = 'INSERT INTO track_list (%s) VALUES %s;'
    
    try:
        cur.execute(insert_statement, (AsIs(','.join(columns)), tuple(values)))
    
    except pg2.IntegrityError:
        print("Duplicate track. Skipping:")
        print(tuple(values))

    con.commit()
    con.close()

In [6]:
def insert_lyrics(lyrics, track_id):
    con, cur = con_cur_to_db()
    
    insert_statement = f"UPDATE track_list SET lyrics = {Json(lyrics)} WHERE track_id = '{track_id}';" 
    
    try:
        cur.execute(insert_statement)

    except pg2.IntegrityError:
        print("Lyrics already added. Skipping:")
        print(track_id)
    
    except:
        print("Upload error. Skipping:")
        print(track_id)
        
    con.commit()
    con.close()

In [7]:
def get_playlist_tracks(category_id='romance', country='us', limit=50, cycles=10, offset=0):

    fin_cycles = 0
    
    while fin_cycles < cycles:
        
        playlists = sp.category_playlists(category_id=category_id, country=country, limit=limit, offset=offset)

        offset = offset + len(playlists['playlists']['items'])

        for playlist in playlists['playlists']['items']:
            playlist_id = playlist['id']
            playlist_name = playlist['name']
            playlist_owner = playlist['owner']['id']

            tracks = sp.user_playlist_tracks(user = playlist_owner, 
                                             playlist_id = playlist_id) 

            for track in tracks['items']:
                try:
                    track_dict = {
                        'track_id' : track['track']['id'],
                        'playlist_id' : playlist_id,
                        'track_name' : track['track']['name'],
                        'artist_name' : track['track']['artists'][0]['name'],
                        'album_name' : track['track']['album']['name'],
                        'playlist_name' : playlist_name,
                        'playlist_owner' : playlist_owner
                    }

                    insert_playlist_info(track_dict)
                
                except TypeError:
                    print(f'Missing track info for track in {playlist_id}. Skipping track.')
                    
            print(f'Uploaded {playlist_name} playlist to SQLdb')

        print(f'Finished uploading {offset} playlists in {category_id} category')
        
        fin_cycles += 1
    
    print(f'Finished uploading cycle: {fin_cycles}')
    print(f'Current offset: {offset}')

In [8]:
def request_song_info(song_title, artist_name):
    search_url = 'https://api.genius.com/search'  
    data = {'q': song_title + ' ' + artist_name}
    headers = {'Authorization': 'Bearer ' + 'Q1xs6KCZB77dD_ZQEub53VCWrqrrNW8xWfMrrrgfj3yNWHLut1Cg-TufGpvVAt2E'}
    
    try:
        response = requests.get(search_url, data=data, headers=headers)
        json = response.json()

    except requests.exceptions.ConnectionError: 
        print(f'*****Connection error for {artist_name}, {song_title}*****')
        print('Retrying connection')
        
        session = requests.Session()
        retry = Retry(connect=3, backoff_factor=0.5)
        adapter = HTTPAdapter(max_retries=retry)
        session.mount('http://', adapter)
        session.mount('https://', adapter)

        session.get(search_url, data=data, headers=headers)
        
    for hit in json['response']['hits']:
        if (song_title.lower() in hit['result']['title'].lower()) & (artist_name.lower() in hit['result']['primary_artist']['name'].lower()): 
            song_url = hit['result']['url']
            print(song_url)
            page = requests.get(song_url)
            html = BeautifulSoup(page.text, 'html.parser')
            lyrics = html.find('div', class_='lyrics').get_text()

            return lyrics

        if (song_title.lower() in hit['result']['title'].lower()) | (artist_name.lower() in hit['result']['primary_artist']['name'].lower()): 
            song_url = hit['result']['url']
            print(song_url)
            page = requests.get(song_url)
            html = BeautifulSoup(page.text, 'html.parser')
            lyrics = html.find('div', class_='lyrics').get_text()

            return lyrics
        
    else:
        print(f'{song_title}, {artist_name} not found')

In [23]:
get_playlist_tracks(category_id='romance', country='us', limit=50, cycles=10, offset=52)

Finished uploading 52 playlists in romance category
Finished uploading 52 playlists in romance category
Finished uploading 52 playlists in romance category
Finished uploading 52 playlists in romance category
Finished uploading 52 playlists in romance category
Finished uploading 52 playlists in romance category
Finished uploading 52 playlists in romance category
Finished uploading 52 playlists in romance category
Finished uploading 52 playlists in romance category
Finished uploading 52 playlists in romance category
Finished uploading cycle: 10
Current offset: 52


In [84]:
query = '''SELECT * FROM track_list;'''
response = execute_query(query, dict_cur=True)
track_df = pd.DataFrame(response)
track_df.set_index('track_id', inplace=True)

In [85]:
track_df.head()

,album_name,artist_name,lyrics,playlist_id,playlist_name,playlist_owner,track_name
track_id,,,,,,,
0h7TlF8gKb61aSm874s3cV,I Can't Tell You How Much It Hurts,moow,\n\nIf your needle is near\nNeedle is near\nYo...,37i9dQZF1DXarebqD2nAVg,Tender,spotify,You'r in My Head
6koowTu9pFHPEcZnACLKbK,Coming Home,Leon Bridges,\n\n[Verse 1]\nBrown skin girl on the other si...,37i9dQZF1DX4adj7PFEBwf,Wedding Bells,spotify,Brown Skin Girl
1JkhKUXAoNivi87ipmV3rp,Back To Love (Deluxe Version),Anthony Hamilton,"\n\n[Verse 1]\nIt's simple, I love it\nHaving ...",37i9dQZF1DX4adj7PFEBwf,Wedding Bells,spotify,Best of Me
51lPx6ZCSalL2kvSrDUyJc,The Search for Everything,John Mayer,\n\n[Intro: Whistling]\n\n[Verse 1]\nA great b...,37i9dQZF1DX4adj7PFEBwf,Wedding Bells,spotify,You're Gonna Live Forever in Me
3vqlZUIT3rEmLaYKDBfb4Q,Songs In The Key Of Life,Stevie Wonder,\n\n[Verse 1]\nIsn't she lovely\nIsn't she won...,37i9dQZF1DX4adj7PFEBwf,Wedding Bells,spotify,Isn't She Lovely


In [63]:
track_df.shape

(2861, 8)

In [64]:
track_df.set_index('track_id', inplace=True)

In [65]:
track_df = track_df[['track_name','artist_name', 'album_name', 'playlist_name', 'playlist_id', 'lyrics']]

In [66]:
track_df.head()

,track_name,artist_name,album_name,playlist_name,playlist_id,lyrics
track_id,,,,,,
0h7TlF8gKb61aSm874s3cV,You'r in My Head,moow,I Can't Tell You How Much It Hurts,Tender,37i9dQZF1DXarebqD2nAVg,\n\nIf your needle is near\nNeedle is near\nYo...
6koowTu9pFHPEcZnACLKbK,Brown Skin Girl,Leon Bridges,Coming Home,Wedding Bells,37i9dQZF1DX4adj7PFEBwf,\n\n[Verse 1]\nBrown skin girl on the other si...
1JkhKUXAoNivi87ipmV3rp,Best of Me,Anthony Hamilton,Back To Love (Deluxe Version),Wedding Bells,37i9dQZF1DX4adj7PFEBwf,"\n\n[Verse 1]\nIt's simple, I love it\nHaving ..."
51lPx6ZCSalL2kvSrDUyJc,You're Gonna Live Forever in Me,John Mayer,The Search for Everything,Wedding Bells,37i9dQZF1DX4adj7PFEBwf,\n\n[Intro: Whistling]\n\n[Verse 1]\nA great b...
3vqlZUIT3rEmLaYKDBfb4Q,Isn't She Lovely,Stevie Wonder,Songs In The Key Of Life,Wedding Bells,37i9dQZF1DX4adj7PFEBwf,\n\n[Verse 1]\nIsn't she lovely\nIsn't she won...


In [115]:
def get_genius_lyrics(df=track_df):
    
    for ix, row in df.iterrows():
        track_id = str(ix)
        song_title = row['track_name'].lower()
        artist_name = row['artist_name']
    
        if (row['lyrics'] == None):
            song_title = re.sub('(^|)((?<=)\s-.+|\s\(.+\)|(\ssoundtrack)|\s(\".+\"))','', song_title) #
            song_title = re.sub('(^|)((\sremastered)|(\sremaster)|(\s\d{4})|(\w+\sversion))','', song_title)
            song_title = re.sub('(^|)((\w+\s+edit)|(\sspotify.+)|(\sat\sspotify.+))','', song_title)
            song_title = re.sub('\s(feat\..+)','', song_title)

            lyrics = request_song_info(song_title=song_title, artist_name=artist_name)

            try:
                if len(lyrics) > 100:
                    insert_lyrics(lyrics, track_id)
                    print(f'Uploaded lyrics for {track_id}, {song_title}')
                    time.sleep(.5)

                else:
                    print(f'*****Lyrics look short. Retry {track_id}, {song_title}*****')

            except TypeError:
                print(f'*****No lyrics for {track_id}, {song_title}*****')
        
    return 'Finished uploading tracks to SQLdb'

In [116]:
get_genius_lyrics(track_df)

https://genius.com/Kendrick-lamar-love-lyrics
Uploaded lyrics for 6PGoSes0D9eUDeeAafB2As, love.
bye bye, ElMari. not found
*****No lyrics for 4s22ihyn2FtBLeKxzXQ3FX, bye bye*****
freshmen, S. Fidelity not found
*****No lyrics for 5a0a6WC2FzLAxNfK7XMYtu, freshmen*****
caught her eyes, C Y G N not found
*****No lyrics for 3PIl4BCEbLuhRax4mgoJ4N, caught her eyes*****
sweetie, Devin Morrison not found
*****No lyrics for 72WU1V0HVg4HWQAk5vdop8, sweetie*****
spazzn, Decap not found
*****No lyrics for 1HY91LimEeoRRmq3EYcUzK, spazzn*****
drumss, J.Robb not found
*****No lyrics for 6qMExjyIOHf4M5rf8Ft2AA, drumss*****
riri thick, Ian Ewing not found
*****No lyrics for 0xT4knFEialXamJIax97Yx, riri thick*****
memory well, David Blazer not found
*****No lyrics for 2UQ1FMQDQuAQKTf8UKohyv, memory well*****
yours and nobody else's, Fallen Roses not found
*****No lyrics for 49VY7zaIgfNI96qdK1Xi2l, yours and nobody else's*****
swell, Ian Ewing not found
*****No lyrics for 1kL4RYBpdW5F33NAqGbxMK, swell**

'Finished uploading tracks to SQLdb'

In [9]:
query = '''SELECT * FROM track_list;'''
response = execute_query(query, dict_cur=True)
track_df = pd.DataFrame(response)
track_df.set_index('track_id', inplace=True)

In [10]:
track_df[['lyrics']].isna().sum()

lyrics    585
dtype: int64

In [11]:
track_df[['lyrics']].isna().sum().values[0] / track_df.shape[0]

0.20447396015379238

In [12]:
track_df[['lyrics', 'track_name', 'artist_name']][track_df['lyrics'].isna()]

,lyrics,track_name,artist_name
track_id,,,
4s22ihyn2FtBLeKxzXQ3FX,None,Bye Bye,ElMari.
5a0a6WC2FzLAxNfK7XMYtu,None,Freshmen,S. Fidelity
3PIl4BCEbLuhRax4mgoJ4N,None,Caught Her Eyes,C Y G N
72WU1V0HVg4HWQAk5vdop8,None,Sweetie,Devin Morrison
1HY91LimEeoRRmq3EYcUzK,None,Spazzn,Decap
6qMExjyIOHf4M5rf8Ft2AA,None,Drumss,J.Robb
0xT4knFEialXamJIax97Yx,None,Riri Thick - Moods Remix,Ian Ewing
2UQ1FMQDQuAQKTf8UKohyv,None,Memory Well,David Blazer
49VY7zaIgfNI96qdK1Xi2l,None,Yours and Nobody Else's,Fallen Roses


In [110]:
song_title = 'LOVE. FEAT. ZACARI.'
artist_name = 'kendrick lamar'

lyrics = request_song_info(song_title=song_title, artist_name=artist_name)

LOVE. FEAT. ZACARI., kendrick lamar not found


In [19]:
len(lyrics)

2608

In [20]:
lyrics

'\n\n[Chorus: Cass]\nI just wanna say fuck you \'til I fuck you again\nI think I\'ve had enough of you, but I don\'t wanna lose a friend\nI\'m gonna love you forever\nThat\'s just my curse\nIt\'s whatever\n\n[Verse 1: Karizma]\nI thought you\'d leave me\nI\'m the bottle, you\'re the genie\nAnd I don\'t think I give a damn\nI felt the beating\nThank god I ate my wheaties\nOr I don\'t know if I could stand\nWe hate to give in, but we\'re hardly living comfortably\nI hate you because you think that you\'re still in love with me\nIt\'s time we drift apart, we\'re blind and in the dark\nI kinda wanna care but it\'s kinda too late to start\nAnd here you are, hitting my line\nHitting rewind on the past for the 50th time\nCome on tell me, what do you think\nWanna lay down, up in the clouds or under the sheets\n\n[Chorus: Cass]\nI just wanna say fuck you \'til I fuck you again\nI think I\'ve had enough of you, but I don\'t wanna lose a friend\nI\'m gonna love you forever\nThat\'s just my curse\

authorization = f'Basic <{client_id}:{client_secret}>'
scope = 'playlist-read-collaborative'
header = {'Authorization':authorization}

response = requests.post('https://accounts.spotify.com/api/token', headers=header, data='grant_type=client_credentials')

response.text

sp.featured_playlists(country='US')

track_name, artist_name, source, source_url
Oh Girl, The Chi-Lites, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
When I Fall In Love, Nat King Cole, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Have I Told You Lately, Van Morrison, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
How Can I Tell You, Cat Stevens, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Alison, Elvis Costello, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Do You Love Me?, Fiddler on the Roof, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Some Enchanted Evening, South Pacific, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Someone to Watch Over Me, Linda Ronstadt, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
A Man and a Woman, Anita Kerr Singers, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Natural Woman, Aretha Franklin, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
All I Want, Joni Mitchell, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
At Last,  Etta James, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
You Make Loving Fun, Fleetwood Mac, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Songbird, Fleetwood Mac, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Just the Way You Are, Billy Joel, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Last Dance, Donna Summer, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Fever, Peggy Lee, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Do You Want to Dance?, Bette Midler, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Sexual Healing, Marvin Gaye, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Crazy, Patsy Cline, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Always on My Mind, Willie Nelson, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Saving All My Love for You, Whitney Houston, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
On the Street Where You Live, From My Fair Lady, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Somewhere, Stephen Sondheim and Leonard Bernstein, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Still in Love With You, Al Green, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Do You Believe in Magic?, The Lovin’ Spoonful, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Just Like a Woman, Bob Dylan, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
P.S. I Love You, The Beatles, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Chelsea Morning, Joni Mitchell, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Girl, The Beatles, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Kiss From a Rose, Seal, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Vision of Love, Mariah Carey, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
With or Without You, U2, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Moon River, Henry Mancini, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Take on Me, a-ha, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Crazy for You, Madonna, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Time After Time, Cyndi Lauper, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Come On Eileen, Dexys Midnight Runners, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Love You Inside Out, Bee Gees, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
You’re the One That I Want, John Travolta and Olivia Newton-John, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
How Deep Is Your Love, The Bee Gees, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
You Don’t Have to Be a Star (To Be in My Show), Marilyn McCoo and Billy Davis Jr., Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Shining Star, Earth Wind & Fire, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Lovin’ You, Minnie Ripperton, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Then Came You, Dionne Warwick and the Spinners, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Can’t Get Enough of Your Love Babe, Barry White, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Feel Like Makin’ Love, Roberta Flack, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Love’s Theme, The Love Unlimited Orchestra, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
The Way We Were, Barbra Streisand, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Midnight Train To Georgia, Gladys Knight, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Let’s Get It On, Marvin Gaye, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
You Are the Sunshine of My Life, Stevie Wonder, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Lean on Me, Bill Withers, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Here, There and Everywhere, The Beatles, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Heart of Gold, Neil Young, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Let’s Stay Together, Al Green, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Ain’t No Mountain High Enough,Diana Ross, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Close to You, The Carpenters, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
The Long and Winding Road, The Beatles, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Sugar Sugar, The Archies, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Higher Love, Steve Winwood, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Love Theme From Romeo and Juliet, Henry Mancini, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
This Guy’s in Love With You, Herb Alpert, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Baby Love, The Supremes, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Strangers in the Night, Frank Sinatra, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Reach Out I’ll Be There, The Four Tops, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
When a Man Loves a Woman, Percy Sledge, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
I Will Always Love You, Dolly Parton/Whitney Houston, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
You’re My Soul and Inspiration The Righteous Brothers, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
My Love, Petula Clark, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
I Got You Babe, Sonny and Cher, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Back in My Arms Again, The Supremes, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Happy Together, The Turtles, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
My Guy,Mary Wells, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Chapel of Love, The Dixie Cups, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
I Will Follow Him, Little Peggy March, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
He’s So Fine, The Chiffons, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Up on the Roof, The Drifters, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Will You Love Me Tomorrow, Carole King, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Stay, Maurice Williams and the Zodiacs, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Save the Last Dance for Me, The Drifters, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Unchained Melody, The Righteous Brothers, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Crazy Little Thing Called Love, Queen, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Sherry, The Four Seasons, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
To Sir With Love, Lulu, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Killing Me Softly With His Song, Roberta Flack, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
You’ve Got a Frien, Carole King, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
All I Have to Do Is Dream, The Everly Brothers, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Bridge Over Troubled Water, Simon & Garfunkel, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Best of My Love, The Emotions, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
I Can’t Stop Loving You, Ray Charles, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Michelle, The Beatles, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Are You Lonesome Tonight?, Elvis Presley, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
The First Time Ever I Saw Your Face, Roberta Flack, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
In My Life, The Beatles, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
You Are So Beautiful, Joe Cocker, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
I Just Called to Say I Love You, Stevie Wonder, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
Wonderful Tonight, Eric Clapton, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
I’ll Be There, The Jackson 5, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,
My Girl, The Temptations, Stereogum, https://www.stereogum.com/4607/the_100_greatest_love_songs_according_to_the_ny_da/franchises/list/,

In [13]:
url = 'https://www.youtube.com/playlist?list=PLgIerhn1S8dPeLzEWOaenUGNYLPIbKI8w'

request = requests.get(url)

In [14]:
soup = BeautifulSoup(request.text, 'lxml')

In [28]:
scrape = soup.findAll('a', attrs={'class':"pl-video-title-link yt-uix-tile-link yt-uix-sessionlink spf-link "})

In [29]:
scrape[0].text.strip()

'Ariana Grande - thank u, next'

## Continue to Notebook 02: Preprocessing